In [1]:
from arguments import ModelParams, ArgumentParser
from depth_images import calibrate_depth
from gaussian_renderer import GaussianModel
from scene import Scene
import matplotlib.pyplot as plt

parser = ArgumentParser()
dataset = ModelParams(parser)
dataset.model_path = ""
dataset.source_path = "../data/icl-nuim/livingroom1/"
dataset.images = "images"
dataset.resolution = -1

dataset.white_background = False
dataset.initialisation = "random"
dataset.eval=True
dataset.num_train_images = 3
gaussians = GaussianModel(dataset.sh_degree)
scene = Scene(dataset, gaussians,shuffle=False)

#calibrate_depth(scene)

Loading Training Cameras
Loading Test Cameras
Number of points at initialisation :  10000


In [2]:
# from PIL import Image
# import numpy as np
# import torch

# img = scene.getTrainCameras()[0].original_image


# vertical_kernel = torch.zeros((3,3,1,7)).float()
# for i in range(3):
#     vertical_kernel[i,i] = torch.tensor([[
#         1,6,15,20,15,6,1
#     ]]).float()/64
# vertical_kernel = vertical_kernel.cuda()
# horizontal_kernel = torch.zeros((3,3,7,1)).float()
# for i in range(3):
#     horizontal_kernel[i,i,:,0] = torch.tensor([
#         1,6,15,20,15,6,1
#     ]).float()/64
# horizontal_kernel = horizontal_kernel.cuda()

# img2 = torch.nn.functional.conv2d(img.unsqueeze(0),vertical_kernel,padding='same').squeeze()
# img2 = torch.nn.functional.conv2d(img2.unsqueeze(0),horizontal_kernel,padding='same').squeeze()

# Image.fromarray((img2.cpu().numpy().transpose((1,2,0))*255).astype(np.uint8))

In [3]:
import open3d as o3d
import numpy as np
from utils.graphics_utils import fov2focal, geom_transform_points

In [4]:
from depth_images import camera_to_pcd

def camera_to_pcd2(camera):
    fx = fov2focal(camera.FoVx,camera.image_width)
    fy = fov2focal(camera.FoVy,camera.image_height)
    intrinics = o3d.camera.PinholeCameraIntrinsic(
        camera.image_width,
        camera.image_height,
        fx,
        fy,
        camera.image_width/2,
        camera.image_height/2,
    )
    print(fx,fy)

    depth_image = 1*camera.depth.numpy()[0].astype(np.float32)
    color_image = (camera.original_image.cpu().numpy().transpose((1,2,0))*255).astype(np.uint8)

    depth_image_o3d = o3d.geometry.Image(np.ascontiguousarray(depth_image))
    color_image_o3d = o3d.geometry.Image(np.ascontiguousarray(color_image))
    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(color_image_o3d,depth_image_o3d,depth_scale=1,depth_trunc=10000,convert_rgb_to_intensity=False)
    
    mat = camera.world_view_transform.cpu().numpy().T
    
    pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image,intrinics,mat)
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=2, max_nn=30))
    return pcd


all_pcds = []
for i,camera in enumerate(scene.getTrainCameras()):
    points,colors = camera_to_pcd(camera)

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(np.ascontiguousarray(points.cpu()))
    pcd.colors = o3d.utility.Vector3dVector(np.ascontiguousarray(colors.cpu()))
    all_pcds.append(pcd)

In [5]:
np.asarray(all_pcds[0].points)

array([[ 0.31057999, -0.57301217, -2.12647963],
       [ 0.31057999, -0.57301217, -2.12647963],
       [ 0.31057999, -0.57301217, -2.12647963],
       ...,
       [ 1.33596754, -0.13438426, -1.8422215 ],
       [ 1.33644664, -0.13438426, -1.84053981],
       [ 1.33692563, -0.13438426, -1.83885825]])

In [6]:
def pairwise_registration(source, target, radius):
    print("Apply point-to-plane ICP")
    icp_fine = o3d.registration.registration_colored_icp(
        source, target, radius,
        np.eye(4),
        o3d.registration.ICPConvergenceCriteria(relative_fitness=1e-6,
                                                          relative_rmse=1e-6,
                                                          max_iteration=50))
    
    transformation_icp = icp_fine.transformation
    information_icp = o3d.registration.get_information_matrix_from_point_clouds(
        source, target, radius,
        icp_fine.transformation)
    return transformation_icp, information_icp


def full_registration(pcds, radius):
    pose_graph = o3d.registration.PoseGraph()
    odometry = np.identity(4)
    pose_graph.nodes.append(o3d.registration.PoseGraphNode(odometry))
    n_pcds = len(pcds)
    for source_id in range(n_pcds):
        for target_id in range(source_id + 1, n_pcds):
            transformation_icp, information_icp = pairwise_registration(
                pcds[source_id], pcds[target_id],radius)
            print("Build o3d.registration.PoseGraph")
            if source_id == 0:  # odometry case
                #odometry = np.dot(transformation_icp, odometry)
                pose_graph.nodes.append(
                    o3d.registration.PoseGraphNode(np.linalg.inv(transformation_icp)))
                pose_graph.edges.append(
                    o3d.registration.PoseGraphEdge(source_id,
                                                             target_id,
                                                             transformation_icp,
                                                             information_icp,
                                                             uncertain=False))
            else:  # loop closure case
                pose_graph.edges.append(
                    o3d.registration.PoseGraphEdge(source_id,
                                                             target_id,
                                                             transformation_icp,
                                                             information_icp,
                                                             uncertain=True))
    return pose_graph

def register_pointclouds(all_pcds):
    print("Full registration ...")
    radius = 0.3
    #with o3d.utility.VerbosityContextManager(
    #        o3d.utility.VerbosityLevel.Debug) as cm:
    pose_graph = full_registration(all_pcds,
                                    radius)

    # print("Optimizing PoseGraph ...")
    # option = o3d.registration.GlobalOptimizationOption(
    #     max_correspondence_distance=radius,
    #     edge_prune_threshold=0.25,
    #     reference_node=0)
    # # with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    # o3d.registration.global_optimization(
    #     pose_graph, o3d.registration.GlobalOptimizationLevenbergMarquardt(),
    #     o3d.registration.GlobalOptimizationConvergenceCriteria(), option)

    for point_id in range(len(all_pcds)):
        print(pose_graph.nodes[point_id].pose)
        all_pcds[point_id].transform(pose_graph.nodes[point_id].pose)

In [7]:
def manual_reg(source, target):
  print("Apply point-to-plane ICP")

  threshold = 0.1
  trans_init = np.eye(4)
  
  # result_icp = o3d.registration.registration_icp(
  #     source, target, threshold, trans_init,
  #     o3d.registration.TransformationEstimationPointToPlane())


  result_icp = o3d.registration.registration_colored_icp(
          source, target, threshold, trans_init,
          o3d.registration.ICPConvergenceCriteria(relative_fitness=1e-6,
                                                          relative_rmse=1e-6,
                                                          max_iteration=50))
  print(result_icp)
  print("Transformation is:")
  print(result_icp.transformation)

  trans = np.linalg.inv(result_icp.transformation)
  target.transform(trans)
  return trans

In [8]:
# register_pointclouds(all_pcds)
# save_pcds = all_pcds

import copy

save_pcds = [copy.deepcopy(all_pcds[i]) for i in range(len(all_pcds))]
transformations = [np.eye(4)]
ref_pcd = save_pcds[0]

# for i in range(1,len(all_pcds)):
#     transformations.append(manual_reg(ref_pcd,save_pcds[i]))
#     ref_pcd = ref_pcd+save_pcds[i]

In [9]:
all_points = []
all_colors = []
for i,pcd in enumerate(save_pcds):
    points = np.array(pcd.points)
    colors = np.array(pcd.colors)
    
    # colors = np.zeros_like(points)
    # colors[:,i%3] = 1.0
    # if i>=3:
    #     colors[:,(i+1)%3] = 1.0
    
    all_points.append(points)
    all_colors.append(colors)
    
# all_points.append(np.stack([camera.camera_center.cpu().numpy() for camera in scene.getTrainCameras()]))
# all_colors.append(np.array([[1.0,0,0]]*len(scene.getTrainCameras())))
    

from depth_images import camera_frustrum_points

cam_frust_points = camera_frustrum_points(scene.getTrainCameras()[0])

all_points.append(cam_frust_points.cpu().numpy())
all_colors.append(np.array([[1.0,0,0]]*len(cam_frust_points)))

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(np.concatenate(all_points))
pcd.colors = o3d.utility.Vector3dVector(np.concatenate(all_colors))
o3d.io.write_point_cloud("test.ply", pcd)
#!cp test.ply /mnt/c/Users/fredr/repos/test.ply

True

In [10]:
[camera.image_name for camera in scene.getTrainCameras()]

['0', '2282', '994']

In [10]:
import torch
import math

def gaussian_kernel_1d(sigma: float, num_sigmas: float = 3.) -> torch.Tensor:
    
    radius = math.ceil(num_sigmas * sigma)
    support = torch.arange(-radius, radius + 1, dtype=torch.float)
    kernel = torch.distributions.Normal(loc=0, scale=sigma).log_prob(support).exp_()
    # Ensure kernel weights sum to 1, so that image brightness is not altered
    return kernel.mul_(1 / kernel.sum())

gaussian_kernel_1d(10,1)

tensor([0.0343, 0.0377, 0.0410, 0.0442, 0.0472, 0.0498, 0.0521, 0.0540, 0.0554,
        0.0562, 0.0565, 0.0562, 0.0554, 0.0540, 0.0521, 0.0498, 0.0472, 0.0442,
        0.0410, 0.0377, 0.0343])

tensor([0.0044, 0.0540, 0.2420, 0.3991, 0.2420, 0.0540, 0.0044])